In [ ]:
! pip uninstall -y transformer_lens
! pip install -U transformers
! pip install git+https://github.com/taufeeque9/TransformerLens/
! pip install git+https://github.com/minyoungg/vqtorch/
! pip install termcolor
! pip install -U accelerate
! pip install -U kaleido


In [1]:
from termcolor import colored
import plotly.express as px
import transformers
import codebook_features
import torch
import evaluate
import numpy as np
import copy
import wandb
import json
import transformer_lens.utils as utils
from collections import namedtuple
from functools import partial
from torch.nn import functional as F
import itertools
from tqdm import tqdm
import pandas as pd

from transformers import (
    GPTNeoXConfig,
    GPTNeoXForCausalLM,
    GPT2TokenizerFast,
    pipeline,
    set_seed,
)
from torch.utils.data import IterableDataset
from codebook_features import models, run_clm, train_toy_model, trainer as cb_trainer   
from codebook_features.utils import *
from codebook_features.toy_utils import *
import os
import math

torch.set_grad_enabled(False)

/data/codebook-features/codebook_features/train_toy_model.py:400: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  @hydra.main(config_path="config", config_name="toy_main")


In [2]:
model_name_or_path = "EleutherAI/pythia-410m-deduped"
# pretrained_path = "/data/outputs/2023-04-25/00-10-18/output_main/" # mlp 70m
# pretrained_path = "/data/outputs/2023-05-08/01-42-10/output_main/" # mlp 410m ccb
pretrained_path = "/shared/outputs/2023-05-04/07-19-37/output_main/" # mlp 410m vcb
device = "cuda"
orig_cb_model = models.wrap_codebook(model_or_path=model_name_or_path, pretrained_path=pretrained_path)
orig_cb_model = orig_cb_model.to(device).eval()
orig_cb_model.disable_logging()
hooked_kwargs = dict(center_unembed=False,fold_value_biases=False,center_writing_weights=True,fold_ln=True,refactor_factored_attn_matrices=False,device=device)
cb_model = models.convert_to_hooked_model(model_name_or_path, orig_cb_model, hooked_kwargs=hooked_kwargs)
# # cb_model = cb_model.model
cb_model.disable_logging()
cb_model = cb_model.to(device).eval()
model = orig_cb_model
tokenizer = cb_model.tokenizer

Using pad_token, but it is not set yet.


Loaded pretrained model EleutherAI/pythia-410m-deduped into HookedTransformer


In [3]:
report_to = "none"
# report_to = "all"
training_args = run_clm.TrainingArguments(
    #     no_cuda=True,
    output_dir="temp_mlp/",
    do_eval=True,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
)

model_args = run_clm.ModelArguments(model_name_or_path=pretrained_path, cache_dir="/shared/.cache/huggingface/")
data_args = run_clm.DataTrainingArguments(
    dataset_name="wikitext", dataset_config_name="wikitext-103-v1", streaming=False,
)

trainer, lm_datasets, last_checkpoint = run_clm.get_trainer_and_dataset(
    model_args,
    data_args,
    training_args,
    model,
    optimizers=(None, None),
)

07/17/2023 04:11:29 - WARNING - codebook_features.run_clm - Process rank: 0, device: cuda:0, n_gpu: 1distributed training: True, 16-bits training: False
07/17/2023 04:11:29 - INFO - codebook_features.run_clm - Training/evaluation parameters TrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
codebook_reg_p=None,
codebook_weight_decay=0.0,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_backend=None,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_delay=0,
eval_steps=None,
evaluation_strategy=no,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
fsdp=[],
fsdp_config={'fsdp_min_num_params': 0, 'xla': False, 'xla_fsdp_grad_ckpt': False},
fsdp_min_num_pa

  0%|          | 0/3 [00:00<?, ?it/s]

07/17/2023 04:11:30 - WARNING - datasets.arrow_dataset - Loading cached processed dataset at /shared/.cache/huggingface/wikitext/wikitext-103-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126/cache-12eba99cec1e5a8c.arrow
07/17/2023 04:11:31 - WARNING - datasets.arrow_dataset - Loading cached processed dataset at /shared/.cache/huggingface/wikitext/wikitext-103-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126/cache-2f9b0caef800bd1c.arrow
07/17/2023 04:11:31 - WARNING - datasets.arrow_dataset - Loading cached processed dataset at /shared/.cache/huggingface/wikitext/wikitext-103-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126/cache-1b8f89a48d258ee8.arrow
07/17/2023 04:11:31 - WARNING - codebook_features.run_clm - The chosen tokenizer supports a `model_max_length` that is longer than the default `block_size` value of 1024. If you would like to use a longer `block_size` up to `tokenizer.model_max_length` you can over

In [4]:
seed = 42
rng = np.random.default_rng(seed)
max_samples = 2000
dataset = lm_datasets["train"].select(rng.choice(len(lm_datasets["train"]), max_samples, replace=False))
tokens = dataset["input_ids"]
tokens_str = []
for i, token in tqdm(enumerate(tokens)):
    tokens_str.append([tokenizer.decode(t) for t in token])
tokens_str = np.array(tokens_str)

from datetime import datetime
import pickle
import glob

trainer.args.report_on = "none"
cb_acts = {}
ft_tkns = {}

def store_cb_activations(key, codebook_ids, codebook_acts=cb_acts):
    assert len(codebook_ids.shape) == 3  # (bs, seq_len, k_codebook)
    if key not in codebook_acts:
        codebook_acts[key] = []
    codebook_acts[key].append(codebook_ids)


output_dir = "/shared/cb_eval_acts/pythia410_mlp*"
dirs = glob.glob(output_dir)
dirs.sort(key=os.path.getmtime)

if len(dirs) > 0:
    output_dir = dirs[-1]
    print("Loading activations from", output_dir)
    with open(f"{output_dir}/cb_acts.pkl", "rb") as f:
        cb_acts = pickle.load(f)
    tokens = np.load(f"{output_dir}/tokens.npy")
    metrics = np.load(f"{output_dir}/metrics.npy", allow_pickle=True)
else:
    orig_cb_model.set_hook_fn(store_cb_activations)
    orig_cb_model.reset_codebook_metrics(), orig_cb_model.reset_hook_kwargs()
    orig_cb_model.enable_logging()
    metrics = trainer.evaluate(dataset)
    for k, v in cb_acts.items():
        cb_acts[k] = np.concatenate(v, axis=0)
    output_dir = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    output_dir = "/shared/cb_eval_acts/pythia410_mlp_" + output_dir
    os.makedirs(output_dir, exist_ok=True)
    np.save(f"{output_dir}/tokens.npy", tokens)
    np.save(f"{output_dir}/metrics.npy", metrics)
    with open(f"{output_dir}/cb_acts.pkl", "wb") as f:
        pickle.dump(cb_acts, f)
    print("Saved activations to", output_dir)

print(metrics)
num_codes = 10_000
cb_at = "mlp"
ccb = ""

2000it [00:07, 281.57it/s]


Loading activations from /shared/cb_eval_acts/pythia410_mlp_2023-07-13_03-31-50
{'eval_loss': 3.046041965484619, 'eval_accuracy': 0.4174623655913979, 'eval_runtime': 1042.2949, 'eval_samples_per_second': 1.919, 'eval_steps_per_second': 0.06, 'eval_multicode_k': 1, 'eval_dead_code_fraction/layer0': 0.0088, 'eval_MSE/layer0': 207.70325073242185, 'eval_input_norm/layer0': 20.950143371582026, 'eval_output_norm/layer0': 23.073701156616206, 'eval_dead_code_fraction/layer1': 0.0, 'eval_MSE/layer1': 245.72139245605462, 'eval_input_norm/layer1': 6.6752554817199705, 'eval_output_norm/layer1': 19.145910171508795, 'eval_dead_code_fraction/layer2': 0.0, 'eval_MSE/layer2': 213.10583129882806, 'eval_input_norm/layer2': 7.069615131378175, 'eval_output_norm/layer2': 18.375904968261718, 'eval_dead_code_fraction/layer3': 0.0, 'eval_MSE/layer3': 194.4261348876953, 'eval_input_norm/layer3': 7.286291618347167, 'eval_output_norm/layer3': 17.63788438415527, 'eval_dead_code_fraction/layer4': 0.0, 'eval_MSE/lay

In [5]:
output_dir

'/shared/cb_eval_acts/pythia410_mlp_2023-07-13_03-31-50'

In [7]:
import multiprocessing as mp
from functools import partial

freq_filter = 5

def process_component(head=None, layer=None):
    layer_dir = f'{output_dir}/layer{layer}'
    os.makedirs(layer_dir, exist_ok=True)
    comp_name = f"layer{layer}_{cb_at}{ccb}{head if ccb else ''}"
    ft_tkns = features_to_tokens(comp_name, cb_acts, num_codes=num_codes)
    with open(f"{layer_dir}/ft_tkns_layer{layer}{'_head'+head if head else ''}.pkl", "wb") as f:
        pickle.dump(ft_tkns, f)
    codes, freqs, acts = print_ft_tkns(ft_tkns,n=3,start=0,stop=10000,tokens=tokens_str,max_examples=100,freq_filter=freq_filter)
    df = pd.DataFrame({"freqs": freqs, "acts": acts}, index=codes)
    df.loc[:, "acts"] = df["acts"].apply(lambda x : x.replace('\n', '\\n'))
    df.to_csv(f"{layer_dir}/code_token_acts_layer{layer}{'_head'+head if head else ''}_filt{freq_filter}.csv", index_label="code")

for layer in tqdm(range(1, cb_model.cfg.n_layers)):
    process_component(layer=layer)


 40%|████      | 805/2000 [00:21<00:31, 38.16it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

100%|██████████| 2000/2000 [00:57<00:00, 34.83it/s]


In [1]:
''

''

In [ ]:
seq = orig_cb_model.generate(do_sample=True, max_length=100, temperature=0.7)
tokenizer.decode(seq[0].tolist(), skip_special_tokens=True)

In [ ]:
cb_model.generate("Resident Evil", max_new_tokens=10, do_sample=False)

In [ ]:
example_prompt = " American"
example_answer = " States"

utils.test_prompt(
    example_prompt,
    example_answer,
    cb_model,
    prepend_bos=False,
    prepend_space_to_answer=False,
    top_k=10,
)

In [ ]:
example_prompt = "Paris is the capital city of"
example_answer = " France"

utils.test_prompt(
    example_prompt,
    example_answer,
    cb_model,
    prepend_bos=False,
    prepend_space_to_answer=False,
    top_k=10,
)

In [ ]:
input_a = "The Italian dish"
input_b = "The football club Manchester"

input_a = cb_model.to_tokens(input_a, prepend_bos=False).to("cuda")
input_b = cb_model.to_tokens(input_b, prepend_bos=False).to("cuda")

logits_a, cache_a = cb_model.run_with_cache(input_a)
logits_b, cache_b = cb_model.run_with_cache(input_b)

print(f"JSD(a, b) = {JSD(logits_a, logits_b)}")

In [ ]:
input_a = "Washington is the capital of The United"
input_b = "London is the capital of The United"

input_a = cb_model.to_tokens(input_a, prepend_bos=False).to("cuda")
input_b = cb_model.to_tokens(input_b, prepend_bos=False).to("cuda")
print(input_a)
print(input_b)
logits_a, cache_a = cb_model.run_with_cache(input_a)
logits_b, cache_b = cb_model.run_with_cache(input_b)

print(f"JSD(a, b) = {JSD(logits_a, logits_b)}")

In [ ]:
input_a = "Paris is"
input_b = "The football club Manchester United is at the top of the league. Manchester"

input_a = cb_model.to_tokens(input_a, prepend_bos=False).to("cuda")
input_b = cb_model.to_tokens(input_b, prepend_bos=False).to("cuda")
print(input_a)
print(input_b)
logits_a, cache_a = cb_model.run_with_cache(input_a)
logits_b, cache_b = cb_model.run_with_cache(input_b)

print(f"JSD(a, b) = {JSD(logits_a, logits_b)}")

In [ ]:
[0,17,19,20]

In [ ]:
pos = -1
# for layer_idx in range(cb_model.cfg.n_layers):
#     n_layers = 1
for layer_idx in range(1):
    n_layers = cb_model.cfg.n_layers
    # n_layers = 1
    cb_at = ( ["mlp"] * cb_model.cfg.n_heads ) * n_layers
    head = (list(range(cb_model.cfg.n_heads))) * n_layers
    # layer = [layer_idx] * len(cb_at)
    layer = range_over_repeat(n_layers, repeat=cb_model.cfg.n_heads)
    # layer = range_over_repeat([0,17,19,20], repeat=cb_model.cfg.n_heads)
    code = [cache_b[get_cb_layer_name(cb_at[i], layer[i], head[i])][0, pos, :] for i in range(len(cb_at))]
    # ind = [0,1,2,3,4,5]
    ind = range(len(cb_at))
    mod_logits, mod_cache = run_with_codes(
        input_a,
        cb_model,
        [code[i] for i in ind],
        [cb_at[i] for i in ind],
        [layer[i] for i in ind],
        [head[i] for i in ind],
        pos=[pos],
    )
    print("Layer:", layer_idx)
    print(logits_to_pred(mod_logits, tokenizer, k=5))
    print(f"JSD(a <- b, b) = {JSD(mod_logits, logits_b, pos=-1)}")


In [ ]:
def patch_in_codes(run_cb_ids, hook, pos, code):
    """Patch in the `code` at `run_cb_ids`."""
    if pos:
        run_cb_ids[:, pos] = code
    else:
        run_cb_ids[:, :] = code
    return run_cb_ids



In [ ]:
find_code_changes(mod_cache, cache_b)

In [ ]:
js_divs = torch.zeros((cb_model.cfg.n_layers, cb_model.cfg.n_heads))
# for layer_idx in tqdm(range(cb_model.cfg.n_layers)):
for layer_idx in tqdm(range(2)):
    for head_idx in range(cb_model.cfg.n_heads):
        # n_layers = cb_model.cfg.n_layers
        cb_at = ["mlp"]
        head = [head_idx]
        layer = [layer_idx] * len(cb_at)
        # layer = range_over_repeat(n_layers, repeat=cb_model.cfg.n_heads)
        # layer = range_over_repeat([0,17,19,20], repeat=cb_model.cfg.n_heads)
        code = [cache_b[get_cb_layer_name(cb_at[i], layer[i], head[i])][0, -1, :] for i in range(len(cb_at))]
        # ind = [0,1,2,3,4,5]
        ind = range(len(cb_at))
        mod_logits, mod_cache = run_with_codes(
            input_a,
            cb_model,
            [code[i] for i in ind],
            [cb_at[i] for i in ind],
            [layer[i] for i in ind],
            [head[i] for i in ind],
            pos=[-1],
        )
        # print("Layer, Head:", layer_idx, head_idx)
        # print(logits_to_pred(mod_logits, tokenizer, k=5))
        # print(f"JSD(a <- b, b) = {JSD(mod_logits, logits_b, pos=-1)}")
        js_divs[layer_idx, head_idx] = JSD(mod_logits, logits_b, pos=-1)

# make the color map range start from 0
imshow(js_divs, xaxis="head", yaxis="layer")



In [ ]:
def range_over_repeat(end_or_list, repeat=1):
    
    if isinstance(end_or_list, int):
        end_or_list = range(end_or_list)
    l = []
    for i in end_or_list:
        l += [i] * repeat
    return l